# Entendendo o Python Data Model

## Conceito

Um modelo de dados é uma abstração que formaliza, organiza ou padroniza a forma como elementos de dados se relacionam entre si e com entidades do mundo real. Em outras palavras, é o modelo de dados que determina a estrutura dos dados,  define como um objeto deve ser construído e como este objeto se comportará tanto como representação de algo, quanto em relação a outros objetos externos ou aos objetos que o compõe.  

O objetivo de um modelo de dados é dar suporte para o desenvolvimento de um sistema, isto é, um conjunto de elementos, concretos ou abstratos, intelectualmente organizado.  

No contexto do modelo de dados do Python, isso significa a existência de uma interface fornecida pela linguagem para lidarmos com os recursos internos da própria linguagem. Ou seja, uma represntação abstrata de si mesma que "formaliza as interfaces dos elementos constituintes da prória linguagem" (Ramalho, 2023), especificando a forma como os objetos são construidos, manipulados e destruídos.

Na prática, é esse conceito que está por trás da afirmação recorrente em muitos livros sobre Python: "Tudo em Python é um objeto". Se tudo em python é um objeto, logo, tudo é a instância de uma classe. Sendo assim, temos uma classe base de onde derivam todo o resto. Essa classe se chama `object`.

A classe `object` é a base para criação de todas as outras classes e fornece um conjunto de atributos e métodos disponíveis implicitamente para todas as classes derivadas. São esses métodos que determinam como o que construímos na linguagem se comunica com os recursos fundamentais da própria linguagem. Chamamos isso de _métodos especiais_. 
Por exemplo:

In [2]:
class Exemple:
    pass
dir(Exemple)

['__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattribute__',
 '__getstate__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__']

A primeira coisa que precisamos saber sobre esses métodos especiais, segundo Ramalho, é que eles foram feitos para serem chamados pelo interpretador Python e não por nós. O interpretador invonca os métodos especiais, muitas vezes com uma sintaxe expecial do tipo `my_object.__*__()`. É essa invocação que realiza as operações básicas sobre objetos. Isto é, nós não escrevemos, por exemplo, `my_object.__len__()`, mas `len(my_object)`. E se `my_object` for a instância de uma classe definida por você, o interpretador chamará o método `__len__` que você implementou. Veja que se chamarmos `Exemple`, o interpretador retorna `__main__.Exemple`.  `__main__` é um dos métodos especiais, mas sua função, assim como os de outros métodos especias serão estudados posteriormente.

In [5]:
Exemple

__main__.Exemple

Essa é a maneira mais simples de observarmos a estrutura do modelo de dados do Python.

## Quando usamos os métodos especiais? 

Ramalho indica que a implementação dos métodos especiais é feita quando "queremos que nossos objetos suportem e interajam com os elementos fundamentais da linguagem". Esses elementos fundamentais são:

- Coleções
- Acesso a atributos
- Iteração (incluíndo iteração assíncrona com `assync for` 
- Sobrecarga (overloading) de operadores
- Invocação de funções e métodos
- Representação e formatação de strings
- Programação assíncrona usando `await`
- Criação e destruição de objetos
- Contextos gerenciados usando instruções `with` ou `async with`

No entanto, ele ressalta que, nosso código não deve conter muitas chamadas diretas à métodos especiais. Isso tem uma razão simples: Como vimos, a chamada a um método especial é implicita (ao menos na maioria das vezes) e existem funções -- desenvolvidas no próprio modelo -- relacionadas a eles para realizarem esse trabalho que, segundo o Ramalho, são mais rápidas (como `len()`, `iter()`, `str()`, por exemplo) do que chamadas diretas. Nos referimos a essas funções como [funções embutidas](https://docs.python.org/pt-br/3.7/library/functions.html) ou funções nativas. Isso porque são funções da própria interface da linguagem que lidam com os tipos embutidos (ou tipos nativos) que, por sua vez, são abstrações criadas a partir de `object` e definem a [hierarquia padrão dos tipos da linguagem](https://docs.python.org/pt-br/3/reference/datamodel.html#the-standard-type-hierarchy). Por tanto, sempre será mais comum implementar os métodos especiais do que usá-los durante o desenvolvimento dos programas.   

A partir do que vimos até aqui, iremos estudar os exemplos expostos por Ramalho ao longo do capítulo em que ele demonstra a implementação e o uso correto dos métodos especiais.

## A ideia por trás do baralho pythônico

O Exemplo 1 do capítulo 1.2 inicia a demonstração do Modelo de Dados do Python através da implementação de dois métodos especiais: `__getitem__` e `__len__`. Para ilustrar, Ramalho, desenvolve um código que representa um baralho como uma sequência de cartas. 

Eu não sou um grande conhecedor de baralhos. Então, não achei trivial dar uma pesquisada rápida sobre o assunto para entender totalmente o código. 

Descobri que o termo "Franch Deck" se refere ao baralho mais comum, o que a maioria de nós conhecemos. Curiosamente ele representa um calendário: são 52 cartas, que correspondem às 52 semanas do ano. Essas cartas são distribuídas por quatro naipes (suits): Copas(hearts ♥), Espadas(spades ♠), Paus(clubs ♣) e Ouros(diamonds ♦). Esses naipes representam as estações do ano e remontam à tradição camponesa da França onde Paus representa os camponeses, Copas o clero, Espadas os militares e Ouros simboliza a burguesia. As cores, vermelha e preta, representam o dia e a noite. Os valores numéricos das cartas, não têm um significado específico, mas possuem um peso hierárquico variando de 2 à 10 (rank) com um "Ás" que, segundos os especialistas, vale 1. Há também as letras J, Q, K, chamadas de cartas chamadas judiciais, que valem 11, 12 e 13, respectivamente e, são simbolizadas por um Valete, uma Dama e um Rei que representam uma espécie de tributo às figuras da alta monarquia.    

Curioso nos depararmos com o fato de que a soma de todas as cartas judiciais do baralho resulta em 12 e a soma do valor numérico de cada carta dá 364, claramente uma referência aos meses e aos dias do ano (dias de um ano não bissexto).

De fato, quase nada disso importa para o nosso exemplo, mas eu achei interessante todo esse significado.

Enfim, o que queremos mesmo, então, é construir um Franch Deck de 52 cards, em um range de 2 à 10, distribuídas em 4 suits cada.

A seguir, a solução do Ramalho:


In [6]:
from collections import namedtuple

In [7]:
Card = namedtuple("Card", ["rank", "suit"])


class FrenchDeck:
    ranks = [str(n) for n in range(2, 11)] + list("JQKA")
    suits = "spades diamonds clubs hearts".split()

    def __init__(self):
        self._cards = [Card(rank, suit) for suit in self.suits 
                                        for rank in self.ranks]

    def __len__(self):
        return len(self._cards)

    def __getitem__(self, position):
        return self._cards[position]

## Entendendo a classe `Card`

Segundo Ramalho, a primeria coisa que precisamos notar é o uso de `namedtuple`, um método da biblioteca padrão `collections`, usado para criar a classe `Card`. Namedtuple constrói uma tupla nomeada. Isto é, uma tupla em que podemos acessar seus elemetos por nome com a **notação de ponto**, a mesma sintaxe que usamos para acessarmos os membros de uma classe. Isso ocorre porque podemos dizer que uma nemedtuple é uma classe que se comporta como uma tupla e sua sintaxe é a seguinte: `namedtuple(typename, field_names, *, rename=False, defaults=None, module=None)`, onde:

- `typename` é o tipo que será retornado. A namedtuple retorna uma subclasse semelhante a uma tupla.
- `field_names` é uma sequência de strings como `['x', 'y']` que também pode ser uma única string separada por caracteres de espaço ou vírgulas, por exemplo `'x y z'`, `'x', 'y', 'z'`. Cada uma das substrings representará um `field_name` de `tipename`.
- `rename` se verdadeiro, nomes de campos inválidos serão substituidos automaticamente por nomes posicionais. Por exemplo `['abc', 'def', 'ghi', 'abc']` é convertido para `['abc', '_1', 'ghi', '_3']`, eliminando a palavra-chave `def` e o nome de campo duplicado `abc`.
- `defaults` pode ser `None` ou um iterável de valor padrão. Como em Python os valores padrão devem vir depois de todos os parâmetros não-padrão, devem estar mais à direita. Para por exemplo, se os nomes dos campos forem `['x', 'y', 'z']` e os padrões são (1, 2), então `x` será um argumento obrigatório, `y` será padrão para 1, e `z` será padrão para 2. 
- `module` se definido, o atributo `__module__` da tupla nomeada é definido para esse valor.

De maneira geral, usamos a tupla nomeada com seus dois primeiros argumentos como no exemplo a seguir tirado da [documetação](https://docs.python.org/3/library/collections.html#collections.namedtuple):

In [8]:
Point = namedtuple('Point', ['x', 'y'])

In [9]:
p = Point(11, y=22) # você pode instanciar usando argumentos posicionais ou nomeados

In [10]:
p[0] + p[1] # são indexados como uma tupla comum

33

In [16]:
x, y = p # os elementos podem ser desempacotados como uma tupla comum
x, y

(11, 22)

In [15]:
p.x + p.y # aqui, a grande diferença: os elementos podem ser pesquisado por seus nomes

33

In [21]:
p.x = 20 # Uma vez instanciado, os atributos são imutáveis como numa tupla

AttributeError: can't set attribute

In [13]:
type_p = p
print(type(type_p)) # o retorno é uma classe

<class '__main__.Point'>


Voltando ao nosso baralho, com uma melhor visão da `nametuple`, podemos dar um rank e uma suit à uma carta:

In [17]:
beer_card = Card("7", "diamonds")
beer_card

Card(rank='7', suit='diamonds')

## Entendendo a classe `FrenchDeck`

O ponto central do código, entretanto, é a classe `FrenchDeck`. Apesar de ser uma classe enxuta, muita coisa acontece nela. 

Primeiro nós temos a soma de duas listas que formam nosso `rank`. A primeira, que estabelece um intervalo numerico (`range`) de 2 à 11 -- que, na prática, resulta em um intervalo de 2 à 10, pois $range = n -1$ -- cujo os numerais são convertido para string. A segunda lista representa as quatro letras das cartas judiciais do baralho. 

Depois temos o `split` de uma string onde cada substring é a representação de uma `suit` do baralho. A função split retorna uma lista contendo cada substring separada.  

A classe `FrenchDeck` é, então, inicializda (`__init__` que será estudado em um outro momento) com uma lista de cartas (`_cards`). Em outras palavras, para cada `rank` e `suit`, de 2 à 10, passamos seus valores como argumentos para nossa classe `Card`, o que resulta em 52 `_cards`.

A partir daqui, entramos no grande objetivo do tópico do capítulo: A implementação dos métodos especiais `__len__` e `__getitem__`. 

Quando implementamos o método `__len__`, permitimos que `FrenchDeck` responda a função `len()` como qulquer outra coleção do Python:

In [18]:
deck = FrenchDeck()

In [19]:
len(deck)

52

Também implementamos `__getitem__`, dando a `FrenchDeck` o poder de se comportar como uma coleção nativa do Python. Isso significa que podemos recuperar elementos da coleção utilizando a **sintaxe de índice** ou se preferir, **notação de colchetes**. Por exemplo, podemos recuperar a primeria e a última carta da coleção:

In [22]:
deck[0]

Card(rank='2', suit='spades')

In [23]:
deck[-1]

Card(rank='A', suit='hearts')

Uma gama de possibilidades se abre para nós porque podemos passar a utilizar bibliotecas desenvolvidas para lidar com coleções do python como, por exemplo, o método `choice` da biblioteca `ramdom`, que nos permite "puxar" cartas aleatóreas do baralho:

In [24]:
from random import choice

In [25]:
choice(deck)

Card(rank='3', suit='hearts')

In [26]:
choice(deck)

Card(rank='J', suit='hearts')

In [27]:
choice(deck)

Card(rank='9', suit='clubs')

Como `__getitem__` usa o operador `[]` de `self._cards`, o baralho também suporta a **sintaxe de fatiamento**:

In [28]:
deck[:3] # busca as primeiras 3 cartas do baralho

[Card(rank='2', suit='spades'),
 Card(rank='3', suit='spades'),
 Card(rank='4', suit='spades')]

In [29]:
deck[12::13] # busca apenas os ases, iniciando no índice 12 e pulando de 13 em 13

[Card(rank='A', suit='spades'),
 Card(rank='A', suit='diamonds'),
 Card(rank='A', suit='clubs'),
 Card(rank='A', suit='hearts')]

Claramente, podemos deduzir que, como o baralho é um coleção, é possível iterar por seus elementos: 

In [30]:
for card in deck:
    print(card)

Card(rank='2', suit='spades')
Card(rank='3', suit='spades')
Card(rank='4', suit='spades')
Card(rank='5', suit='spades')
Card(rank='6', suit='spades')
Card(rank='7', suit='spades')
Card(rank='8', suit='spades')
Card(rank='9', suit='spades')
Card(rank='10', suit='spades')
Card(rank='J', suit='spades')
Card(rank='Q', suit='spades')
Card(rank='K', suit='spades')
Card(rank='A', suit='spades')
Card(rank='2', suit='diamonds')
Card(rank='3', suit='diamonds')
Card(rank='4', suit='diamonds')
Card(rank='5', suit='diamonds')
Card(rank='6', suit='diamonds')
Card(rank='7', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='A', suit='diamonds')
Card(rank='2', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='8', sui

Por que não iterar de trás para frente?

In [31]:
for card in reversed(deck):
    print(card)

Card(rank='A', suit='hearts')
Card(rank='K', suit='hearts')
Card(rank='Q', suit='hearts')
Card(rank='J', suit='hearts')
Card(rank='10', suit='hearts')
Card(rank='9', suit='hearts')
Card(rank='8', suit='hearts')
Card(rank='7', suit='hearts')
Card(rank='6', suit='hearts')
Card(rank='5', suit='hearts')
Card(rank='4', suit='hearts')
Card(rank='3', suit='hearts')
Card(rank='2', suit='hearts')
Card(rank='A', suit='clubs')
Card(rank='K', suit='clubs')
Card(rank='Q', suit='clubs')
Card(rank='J', suit='clubs')
Card(rank='10', suit='clubs')
Card(rank='9', suit='clubs')
Card(rank='8', suit='clubs')
Card(rank='7', suit='clubs')
Card(rank='6', suit='clubs')
Card(rank='5', suit='clubs')
Card(rank='4', suit='clubs')
Card(rank='3', suit='clubs')
Card(rank='2', suit='clubs')
Card(rank='A', suit='diamonds')
Card(rank='K', suit='diamonds')
Card(rank='Q', suit='diamonds')
Card(rank='J', suit='diamonds')
Card(rank='10', suit='diamonds')
Card(rank='9', suit='diamonds')
Card(rank='8', suit='diamonds')
Card(r

Ramalho também demonstra que a iteração é implícita. Isto é "se uma coleção não fornece um método `__contains__`, o operador `is` realiza uma busca sequencial:  

In [32]:
Card('Q', 'hearts') in deck

True

In [33]:
Card('7', 'beasts') in deck

False

Por fim, podemos ordenar as cartas do baralho. Uma maneira de ordenar um baralho é por seu valor numérico (sendo os ases os mais altos) e depois pelo naipe na ordem espadas (o mais alto), copas, ouros e paus (o mais baixo). Para fazer isso, Ramalho desenvolve uma função que devolve 0 para o 2 de paus e 51 para o às de espadas:



In [34]:
suit_vaules = dict(spades=3, hearts=2, diamonds=1, clubs=0)

def spades_high(card):
    rank_value = FrenchDeck.ranks.index(card.rank)
    return rank_value * len(suit_vaules) + suit_vaules[card.suit]

Agora, podemos listar em ordem crescente:

In [35]:
for card in sorted(deck, key=spades_high):
    print(card)

Card(rank='2', suit='clubs')
Card(rank='2', suit='diamonds')
Card(rank='2', suit='hearts')
Card(rank='2', suit='spades')
Card(rank='3', suit='clubs')
Card(rank='3', suit='diamonds')
Card(rank='3', suit='hearts')
Card(rank='3', suit='spades')
Card(rank='4', suit='clubs')
Card(rank='4', suit='diamonds')
Card(rank='4', suit='hearts')
Card(rank='4', suit='spades')
Card(rank='5', suit='clubs')
Card(rank='5', suit='diamonds')
Card(rank='5', suit='hearts')
Card(rank='5', suit='spades')
Card(rank='6', suit='clubs')
Card(rank='6', suit='diamonds')
Card(rank='6', suit='hearts')
Card(rank='6', suit='spades')
Card(rank='7', suit='clubs')
Card(rank='7', suit='diamonds')
Card(rank='7', suit='hearts')
Card(rank='7', suit='spades')
Card(rank='8', suit='clubs')
Card(rank='8', suit='diamonds')
Card(rank='8', suit='hearts')
Card(rank='8', suit='spades')
Card(rank='9', suit='clubs')
Card(rank='9', suit='diamonds')
Card(rank='9', suit='hearts')
Card(rank='9', suit='spades')
Card(rank='10', suit='clubs')
Ca

## Resumo do conceito geral 

O Modelo de Dados do Python é uma representação abstrata da própria linguagem. Ele fornece uma interface sofisticada para dar acesso aos seus próprios recursos fundamentais. Esse alto nível de abstração permite uma grande flexibilidade na forma como os objetos são construidos e manipulados e faz do Python uma linguagem extremamente idiomática e legível, além de ser o pilar da sua filosofia e sucesso como linguagem de programação. 

Conseguimos observar essas características através da implementação de uma classe simples que representa um baralho. Ramalho demonstra como a implementação dos métodos especiais `__getitem__` e `__len__` deram à classe `FrenchDeck` a possibilidade de se comportar como uma seqência padrão do Python fornecendo acesso aos recurso centrais para tanto (iteração e fatiamento por exemplo). Isso proporcionou, segundo o autor, ao menos duas grandes vantagens:

1. Os usuários da classe `FrenchDeck` não precisam memorizar nomes arbitrários de métodos para operações comuns, como descobrir, por exemplo, o número de itens na coleção (em outras linguagens poderíamos ter um `.size()` ou um `.length()`?).

2. É mais fácil aproveitar a rica biblioteca padrão do Python ao invés de reinventar a roda, como no caso do uso de `random.choice`, `sorted`, `reverse`, por exemplo.

Nos próximos tópicos, 1.3 e 1.4, iremos nos aprofundar nos usos mais importantes dos métodos especiais e nos métodos epeciais mais usados.